In [19]:
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text
import gensim

In [20]:
data=open('CBOW.txt','r')
corona_data = [text for text in data if text.count(' ')>=2]
corona_data

['The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \n',
 'Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \n',
 'The reproductive number – the number of secondary infections generated from one infected individual – is understood to b

In [21]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corona_data)
corona_data=tokenizer.texts_to_sequences(corona_data)

In [22]:
window_size=2
total_vaca=sum(len(x) for x in corona_data)
word_count=len(tokenizer.word_index)+1

In [23]:
def cbow_model(data,window_size,total_vaca):
  total_length=window_size*2
  for text in data:
    text_len=len(text)
    for idx, word in enumerator(text):
      context_word=[]
      target=[]
      begin=idx-window_size
      end=idx+window_size
      context_word.append([text[i] for i in range(begin, end) if 0 <= i < text_len and i != idx])
      target.append(word)
      contextual = sequence.pad_sequences(context_word, total_length=total_length)
      final_target = np_utils.to_categorical(target, total_vocab)
      yield(contextual, final_target)



In [24]:

model = Sequential()
model.add(Embedding(input_dim=total_vaca, output_dim=100, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
model.add(Dense(total_vaca, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
for i in range(10):
    cost = 0
    for x, y in cbow_model(data, window_size, total_vaca):
        cost += model.train_on_batch(contextual, final_target)
    print(i, cost)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0


In [25]:
dimensions=100
vect_file = open('/content/vectors.txt' ,'w')
vect_file.write('{} {}\n'.format(total_vaca,dimensions))

8

In [26]:
weights = model.get_weights()[0]
for text, i in tokenizer.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()

In [27]:
cbow_output = gensim.models.KeyedVectors.load_word2vec_format('/content/vectors.txt', binary = False, limit=100)


In [28]:
cbow_output.most_similar(positive=['virus'])


[('6', 0.2983115017414093),
 ('infected', 0.2218937873840332),
 ('while', 0.19295375049114227),
 ('the', 0.14164185523986816),
 ('is', 0.14030338823795319),
 ('further', 0.14011487364768982),
 ('important', 0.13778285682201385),
 ('understood', 0.1353565752506256),
 ('context', 0.11918006092309952),
 ('successive', 0.11898145079612732)]